# Задача № 2

С помощью Python сформировать последовательности источников визитов пользователя, которые привели к транзакции.

- Данные взяты из Google BigQuery (bigquery-public-data.google_analytics_sample-ga_sessions), период запроса - 2 недели.
- Источник определяется как связка utm_source + utm_medium. 
- Транзакция определяется, как hits.transaction.transactionId и соответствует регулярному выражению `.*\d+.*`

Пример:
- Пользователь, который совершил визит с рекламы Яндекса, а затем с рекламы Google и совершил транзакцию будет иметь цепочку
yandex_cpc => google_cpc
- Пользователь совершил переход из поиска Google, а затем из поиска Яндекс. Транзакций не совершал. Цепочка для него отсутствует.

**Задача** - получить 2 результирующие таблицы:
1. Для каждой транзакции сформирована цепочка касаний
2. Топ-10 самых популярных цепочек касаний. Популярность определяется на основе количества таких цепочек в датасете.

## Запрос к BigQuery

```
SELECT DISTINCT fullVisitorId, visitId, trafficSource.source, trafficSource.medium, h.transaction.transactionId
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_2017*`
CROSS JOIN UNNEST(hits) as h
WHERE  _TABLE_SUFFIX BETWEEN '0719'AND '0801'
ORDER BY fullVisitorId, visitId
```

## Предобработка данных

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('bquxjob_3d4ff71c_184a441cd7c.csv', dtype={
    'fullVisitorId': str ,
    'visitId': np.int64,
    'source': str,
    'medium': str,
    'transactionId': str 
})

df.head()

,fullVisitorId,visitId,source,medium,transactionId
0,0000085059828173212,1500505105,(direct),(none),NaN
1,0000436683523507380,1500504900,(direct),(none),NaN
2,0000436683523507380,1500989127,(direct),(none),NaN
3,0000572434142265465,1500605115,(direct),(none),NaN
4,000170187170673177,1501020630,(direct),(none),NaN


In [3]:
# добавляем столбец, где определяем, соответствует ли 'transactionId' регулярному выражению

def is_transactionId_valid(transactionId):
    
    '''
     возвращает значение True, если transactionId соответствует регулярному выражению
     и значение False в остальных случаях.
    '''
    if pd.notnull(transactionId):
        if re.match('.*\d+.*', str(transactionId)):
            return True
    return False

df['transactionId_valid'] = df['transactionId'].apply(is_transactionId_valid)

In [4]:
# оставляем в датасете данные только по тем пользователям, у которых есть заданные транзакции

# формируем список пользователей, совершивших транзакции

visitor_ids = df[df['transactionId_valid'] == True]['fullVisitorId'].unique().tolist()

# фильтруем исходный датасет

df = df[df['fullVisitorId'].isin(visitor_ids)]

In [5]:
# сливаем столбцы 'source' и 'medium' в одну

df['source+medium'] = df['source'] + '_' + df['medium']

df['source+medium'] = df['source+medium'].apply(lambda x: x.replace("(direct)_(none)", "direct"))

df = df.drop(['source', 'medium', 'transactionId_valid'], axis=1)

In [6]:
# в столбце 'transactionId' нулевые значения заполняем номером транзакции, к которой привели визиты

df = df.bfill(axis ='rows').drop_duplicates()

## Вычисление цепочек касаний

In [7]:
# определяем цепочки касаний для пользователя

# создаем функцию, которая соединяет источники в цепочку касаний для каждой транзакции

def chains(rows):
    
    # задаем начальное значение в цепочке
    chain_str = ''
 
    for row in rows:
        if chain_str != '':
            chain_str += ' => ' # задаем разделитель источников в цепочке       

        chain_str += row
    
    return chain_str

In [8]:
# выводим таблицу с транзакциями и соответствующими цепочками касаний

data_chains = df.groupby(['transactionId'], as_index=False).agg(chains).drop('fullVisitorId', axis=1)

data_chains.columns = ['transactionId', 'chain']

pd.set_option('display.max_colwidth', None)
data_chains.head(20)

,transactionId,chain
0,ORD201707192235,direct
1,ORD201707192281,google_organic => mail.google.com_referral => google_cpc
2,ORD201707192288,direct
3,ORD201707192330,direct => direct => direct => google_cpc
4,ORD201707192331,direct => direct
5,ORD201707192337,direct
6,ORD201707192346,direct
7,ORD201707192352,direct
8,ORD201707192353,direct
9,ORD201707192355,direct


## Топ-10 цепочек касаний

In [9]:
# определяем Топ-10 самых популярных цепочек касаний

chains_top_10 = (
    data_chains
    .groupby('chain', as_index=False)
    .agg({'transactionId': 'count'})
)

chains_top_10.columns = ['chain', 'chain_count']

chains_top_10 = chains_top_10.sort_values(by='chain_count', ascending=False).reset_index(drop=True)

chains_top_10 = chains_top_10.loc[0:10] # выводим 11 строк, так как на 10м месте 2 цепочки с одинаковым количеством

chains_top_10

,chain,chain_count
0,direct,156
1,direct => direct,68
2,google_organic,58
3,direct => direct => direct,31
4,direct => direct => direct => direct,21
5,google_organic => google_organic,17
6,direct => google_organic,13
7,google_cpc,12
8,google_organic => direct,11
9,google_cpm,6
